In [1]:
import sys
import time
import datetime

In [2]:
TOPIC_Step2_NAME="User-Raw-Data"
KAFKA_SERVER="kafka-broker:29092"

In [3]:
import os
# https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html

# setup arguments
# os.environ['PYSPARK_SUBMIT_ARGS']='--packages  org.zalando:spark-json-schema_2.12:0.6.3 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS']='--packages  org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Step3_4-Count-Provinces") \
    .config("spark.executor.memory", "500mb") \
    .config("spark.executor.cores","1") \
    .config("spark.cores.max", "1") \
    .config("spark.sql.session.timeZone", "Asia/Tehran") \
    .getOrCreate()    
    

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3ae5cfe3-9e16-4bc0-a8cc-57f8109918cf;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 1980ms :: artifacts dl 15ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central in [default]

In [4]:
spark.sparkContext.setLogLevel("ERROR")


In [5]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_SERVER) \
  .option("subscribe", TOPIC_Step2_NAME) \
  .option("startingOffsets", "earliest") \
  .option("kafka.group.id", "step3_4-Count-Province")\
  .load()

In [6]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
# sample json must be in jsonl format
# cat user.json | jq -c
df2 = spark.read.json("/opt/spark-data/user.json")

In [8]:
df2.printSchema()

root
 |-- cell: string (nullable = true)
 |-- dob: struct (nullable = true)
 |    |-- age: long (nullable = true)
 |    |-- date: string (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- id: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- value: string (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- coordinates: struct (nullable = true)
 |    |    |-- latitude: string (nullable = true)
 |    |    |-- longitude: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- postcode: long (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- street: struct (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- number: long (nullable = true)
 |    |-- timezone: struct (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- offset: string (nullable = true)
 |-- login: s

In [9]:
user_schema = df2.schema
user_schema

StructType(List(StructField(cell,StringType,true),StructField(dob,StructType(List(StructField(age,LongType,true),StructField(date,StringType,true))),true),StructField(email,StringType,true),StructField(gender,StringType,true),StructField(id,StructType(List(StructField(name,StringType,true),StructField(value,StringType,true))),true),StructField(location,StructType(List(StructField(city,StringType,true),StructField(coordinates,StructType(List(StructField(latitude,StringType,true),StructField(longitude,StringType,true))),true),StructField(country,StringType,true),StructField(postcode,LongType,true),StructField(state,StringType,true),StructField(street,StructType(List(StructField(name,StringType,true),StructField(number,LongType,true))),true),StructField(timezone,StructType(List(StructField(description,StringType,true),StructField(offset,StringType,true))),true))),true),StructField(login,StructType(List(StructField(md5,StringType,true),StructField(password,StringType,true),StructField(salt

In [10]:
userStringDF = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")


In [11]:
userDF = userStringDF.select(from_json(col("value"), user_schema).alias("data")).select("data.*")
# tweetsDF = tweetsDF.withColumn("timestamp", unix_timestamp("sendTime", "yyyy-MM-dd'T'HH:mm:ssz").cast('timestamp')) \
#             .withColumn("persianYear", tweetsDF['sendTimePersian'].substr(0, 4)) \
#             .withColumn("persianMonth", tweetsDF['sendTimePersian'].substr(6, 2)) \
#             .withColumn("persianDay", tweetsDF['sendTimePersian'].substr(9, 2))


In [12]:
# The User Defined Function (UDF)
# Create a timestamp from the current time and return it
import jdatetime
def add_timestamp():
         ts = time.time()
         timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
         return timestamp
def add_persian_timestamp():
         persian_timestamp = jdatetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
         return persian_timestamp

In [13]:
jdatetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'1400-07-14 13:52:21'

In [14]:
# Register the UDF
# Set the return type to be a String
# A name is assigned to the registered function 
add_timestamp_udf = udf(add_timestamp, StringType())
add_persian_timestamp_udf = udf(add_persian_timestamp, StringType())

In [15]:
userDF = userDF.withColumn("timestamp", add_timestamp_udf())
userDF = userDF.withColumn("persian_timestamp", add_persian_timestamp_udf())

In [ ]:
windowedStateCounts = userDF.select("persian_timestamp","location.state")\
                                     .groupBy(\
                                            window(userDF.persian_timestamp, \
                                                    "1 hours", \
                                                    "30 minutes"),\
                                            "state")\
                                     .count()\
                                     .orderBy([col("window").desc(),col("count").desc() ])
                                     # .orderBy(col("count").desc())

#                                      .filter(col('count')>2) \
#                                      
                                     

query = windowedStateCounts\
        .writeStream\
        .outputMode("complete")\
        .format("console")\
        .option("truncate", "false")\
        .option("checkpointLocation", "/opt/spark/spark-apps/")\
        .option("numRows", 30)\
        .start()\
        .awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+-------------------+-----+
|window                                    |state              |count|
+------------------------------------------+-------------------+-----+
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|یزد                |148  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|گلستان             |140  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|سمنان              |137  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|کرمان              |132  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|آذربایجان شرقی     |132  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|کهگیلویه و بویراحمد|131  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|آذربایجان غربی     |128  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|مازندران           |126  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|زنجان              |125  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|اصفهان 

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+-------------------+-----+
|window                                    |state              |count|
+------------------------------------------+-------------------+-----+
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|یزد                |158  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|سمنان              |145  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|گلستان             |144  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|کرمان              |139  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|آذربایجان شرقی     |138  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|آذربایجان غربی     |135  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|کهگیلویه و بویراحمد|134  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|زنجان              |132  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|مازندران           |131  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|همدان  

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+-------------------+-----+
|window                                    |state              |count|
+------------------------------------------+-------------------+-----+
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|یزد                |161  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|گلستان             |150  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|سمنان              |148  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|آذربایجان شرقی     |142  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|کرمان              |140  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|کهگیلویه و بویراحمد|137  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|آذربایجان غربی     |137  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|زنجان              |136  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|مازندران           |132  |
|{1400-07-14 13:25:44, 1400-07-14 14:25:44}|همدان  

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+-------------------+-----+
|window                                    |state              |count|
+------------------------------------------+-------------------+-----+
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|گیلان              |7    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خراسان رضوی        |6    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خراسان شمالی       |6    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|بوشهر              |5    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خوزستان            |5    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|قم                 |5    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|کرمانشاه           |4    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|همدان              |3    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|کرمان              |3    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|مازندرا

-------------------------------------------
Batch: 4
-------------------------------------------
+------------------------------------------+-------------------+-----+
|window                                    |state              |count|
+------------------------------------------+-------------------+-----+
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خوزستان            |9    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|بوشهر              |8    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خراسان رضوی        |8    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|چهارمحال و بختیاری |8    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|گیلان              |8    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|همدان              |7    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خراسان شمالی       |7    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|اردبیل             |6    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|قزوین              |6    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|یزد    

-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+-------------------+-----+
|window                                    |state              |count|
+------------------------------------------+-------------------+-----+
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|یزد                |11   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|همدان              |10   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خوزستان            |10   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خراسان رضوی        |9    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خراسان شمالی       |9    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|کهگیلویه و بویراحمد|9    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|بوشهر              |8    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|کرمان              |8    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|لرستان             |8    |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|چهارمحا

-------------------------------------------
Batch: 6
-------------------------------------------
+------------------------------------------+-------------------+-----+
|window                                    |state              |count|
+------------------------------------------+-------------------+-----+
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|یزد                |14   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|همدان              |13   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خوزستان            |12   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|چهارمحال و بختیاری |12   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خراسان شمالی       |12   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|کرمان              |11   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|خراسان رضوی        |10   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|اردبیل             |10   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|کهگیلویه و بویراحمد|10   |
|{1400-07-14 13:55:44, 1400-07-14 14:55:44}|بوشهر  

In [ ]:
spark.stop()